In [17]:
import sqlite3
import pandas as pd

# Connect to the chinook database
con = sqlite3.connect('/Users/samadjon/Phyton_homeworks-7/Phyton_homeworks/lesson-17/chinook.db')

# Load customers table
df_customers = pd.read_sql_query("SELECT * FROM customers", con)

# Load invoices table
df_invoices = pd.read_sql_query("SELECT * FROM invoices", con)

# Close the connection
con.close()

# Merge DataFrames on CustomerId (inner join by default)
df_merged = pd.merge(df_customers, df_invoices, on='CustomerId', how='inner')

# Count total invoices per customer
invoice_counts = df_merged.groupby(['CustomerId', 'FirstName', 'LastName']).size().reset_index(name='TotalInvoices')

# Display the result
invoice_counts.head(10)


,CustomerId,FirstName,LastName,TotalInvoices
0,1,Luís,Gonçalves,7
1,2,Leonie,Köhler,7
2,3,François,Tremblay,7
3,4,Bjørn,Hansen,7
4,5,František,Wichterlová,7
5,6,Helena,Holý,7
6,7,Astrid,Gruber,7
7,8,Daan,Peeters,7
8,9,Kara,Nielsen,7
9,10,Eduardo,Martins,7


In [19]:
import pandas as pd

# Load the movie.csv file
df_movie = pd.read_csv('/Users/samadjon/Phyton_homeworks-7/Phyton_homeworks/lesson-16/homework/movie.csv')

# Create the first DataFrame with director_name and color
df_color = df_movie[['director_name', 'color']]

# Create the second DataFrame with director_name and num_critic_for_reviews
df_critics = df_movie[['director_name', 'num_critic_for_reviews']]


In [20]:
# Left Join on director_name
left_join = pd.merge(df_color, df_critics, on='director_name', how='left')

# Full Outer Join on director_name
outer_join = pd.merge(df_color, df_critics, on='director_name', how='outer')


In [21]:
# Count the number of rows in each joined DataFrame
print("Left Join Row Count:", len(left_join))
print("Full Outer Join Row Count:", len(outer_join))


Left Join Row Count: 30300
Full Outer Join Row Count: 30300


In [23]:
import pandas as pd

# Load titanic dataset
df_titanic = pd.read_excel('/Users/samadjon/Phyton_homeworks-7/Phyton_homeworks/lesson-16/homework/titanic.xlsx')

# Group by Pclass and calculate the required aggregations
titanic_grouped = df_titanic.groupby('Pclass').agg(
    average_age=('Age', 'mean'),
    total_fare=('Fare', 'sum'),
    passenger_count=('PassengerId', 'count')
).reset_index()

# Display the results
print(titanic_grouped)


   Pclass  average_age  total_fare  passenger_count
0       1    38.233441  18177.4125              216
1       2    29.877630   3801.8417              184
2       3    25.140620   6714.6951              491


In [25]:


# Group by color and director_name
movie_grouped = df_movie.groupby(['color', 'director_name']).agg(
    total_critic_reviews=('num_critic_for_reviews', 'sum'),
    average_duration=('duration', 'mean')
).reset_index()

# Display the top 10 rows
print(movie_grouped.head(10))


             color     director_name  total_critic_reviews  average_duration
0  Black and White    Akira Kurosawa                 153.0             202.0
1  Black and White    Aleksey German                 121.0             177.0
2  Black and White      Alex Garland                 489.0             108.0
3  Black and White   Alexander Payne                 433.0             115.0
4  Black and White  Alfred Hitchcock                 434.0             119.0
5  Black and White        Allan Dwan                  40.0             109.0
6  Black and White  Andrei Tarkovsky                 144.0             115.0
7  Black and White   Andrew Bujalski                  52.0             109.0
8  Black and White     Andrew Currie                 140.0              93.0
9  Black and White      Andrew Davis                 106.0             117.0


In [28]:


# Define custom function
def classify_age(age):
    if pd.isnull(age):
        return 'Unknown'
    return 'Child' if age < 18 else 'Adult'

# Apply the function
df_titanic['Age_Group'] = df_titanic['Age'].apply(classify_age)

# Display the updated DataFrame
print(df_titanic[['Age', 'Age_Group']].head(10))

    Age Age_Group
0  22.0     Adult
1  38.0     Adult
2  26.0     Adult
3  35.0     Adult
4  35.0     Adult
5   NaN   Unknown
6  54.0     Adult
7   2.0     Child
8  27.0     Adult
9  14.0     Child


In [32]:

# Load employee.csv
df_employee = pd.read_csv('/Users/samadjon/Phyton_homeworks-7/Phyton_homeworks/lesson-16/homework/employee.csv')

# Normalize salary within each department using groupby and transform
df_employee['Normalized_Salary'] = df_employee.groupby('DEPARTMENT')['BASE_SALARY'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

# Display result
print(df_employee.head(10))



   UNIQUE_ID                            POSITION_TITLE  \
0          0               ASSISTANT DIRECTOR (EX LVL)   
1          1                         LIBRARY ASSISTANT   
2          2                            POLICE OFFICER   
3          3                         ENGINEER/OPERATOR   
4          4                               ELECTRICIAN   
5          5                     SENIOR POLICE OFFICER   
6          6                                  ENGINEER   
7          7                                 CARPENTER   
8          8  DEPUTY ASSISTANT DIRECTOR (EXECUTIVE LEV   
9          9            AIRPORT OPERATIONS COORDINATOR   

                       DEPARTMENT  BASE_SALARY                       RACE  \
0     Municipal Courts Department     121862.0            Hispanic/Latino   
1                         Library      26125.0            Hispanic/Latino   
2   Houston Police Department-HPD      45279.0                      White   
3   Houston Fire Department (HFD)      63166.0       

In [33]:
# Load movie.csv


# Define classification function
def classify_duration(duration):
    if pd.isnull(duration):
        return 'Unknown'
    if duration < 60:
        return 'Short'
    elif duration <= 120:
        return 'Medium'
    else:
        return 'Long'

# Apply the function
df_movie['Length_Category'] = df_movie['duration'].apply(classify_duration)

# Show some examples
print(df_movie[['duration', 'Length_Category']].head(10))


   duration Length_Category
0     178.0            Long
1     169.0            Long
2     148.0            Long
3     164.0            Long
4       NaN         Unknown
5     132.0            Long
6     156.0            Long
7     100.0          Medium
8     141.0            Long
9     153.0            Long


In [34]:

# Create the pipeline
df_pipeline = (
    df_titanic[df_titanic['Survived'] == 1]  # Filter survivors
    .copy()  # To avoid SettingWithCopyWarning
)

# Fill missing Age values with mean
mean_age = df_pipeline['Age'].mean()
df_pipeline['Age'].fillna(mean_age, inplace=True)

# Create new column Fare_Per_Age
df_pipeline['Fare_Per_Age'] = df_pipeline['Fare'] / df_pipeline['Age']

# Display the result
print(df_pipeline[['Survived', 'Age', 'Fare', 'Fare_Per_Age']].head(10))


    Survived       Age     Fare  Fare_Per_Age
1          1  38.00000  71.2833      1.875876
2          1  26.00000   7.9250      0.304808
3          1  35.00000  53.1000      1.517143
8          1  27.00000  11.1333      0.412344
9          1  14.00000  30.0708      2.147914
10         1   4.00000  16.7000      4.175000
11         1  58.00000  26.5500      0.457759
15         1  55.00000  16.0000      0.290909
17         1  28.34369  13.0000      0.458656
19         1  28.34369   7.2250      0.254907


/var/folders/js/ymx9pdp10wxggvhltkc1ykp80000gn/T/ipykernel_776/2248169331.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_pipeline['Age'].fillna(mean_age, inplace=True)
